In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import os

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

from gaussfit import *

## Functions for channel mapping

In [ ]:
def readSqlitedb(database="/cvmfs/icarus.opensciencegrid.org/products/icarus/icarus_data/v09_62_00/icarus_data/database/ChannelMapICARUS.db", table="pmt_placements"):

    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(database)
    df = pd.read_sql_query("SELECT * from {}".format(table), con)
    con.close()

    return df

def channel_to_PMTid(channels):
    
    geo = readSqlitedb()
    if np.isscalar(channels):
        pmt_id = geo[geo.channel_id==channels].pmt_id.values[0]
        return pmt_id
    else:
        pmt_ids = [ geo[geo.channel_id==ch].pmt_id.values[0] for ch in channels ] 
        return pmt_ids
    
def PMTid_to_channel(pmt_ids):
    
    geo = readSqlitedb()
    if np.isscalar(pmt_ids):
        channel = geo[geo.pmt_id==pmt_ids].channel_id.values[0]
        return channel
    else:
        channels = [ geo[geo.pmt_id==pmt].channel_id.values[0] for pmt in pmt_ids ] 
        return channels
    
def getCryo(channel):
    board = geo[geo.channel_id==channel].digitizer_label.values[0]
    wall, pos, num = board.split("-")
    if wall[0] == "W":
        return 1
    elif wall[0] == "E":
        return 0
    
def readPlacements(file="/icarus/data/users/mvicenzi/pmt-calibration/input/pmt_positions.csv"):
    geo = pd.read_csv(file,sep=",")
    geo.drop(columns=["entry","subentry"],inplace=True)
    return geo

def getX(ch):
    geo = readPlacements()
    return geo["pmt_x"].iloc[ch]

def getY(ch):
    geo = readPlacements()
    return geo["pmt_y"].iloc[ch]

def getZ(ch):
    geo = readPlacements()
    return geo["pmt_z"].iloc[ch]

In [ ]:
from scipy.optimize import curve_fit

def line(x, a, b ):
    return a*x+b

## Functions for data loading

In [ ]:
# Get the timestamp
def getTimestamp(file):
    
    buff=file.split('_')
    timestamp = int(buff[-1].split('.')[0])
    
    return timestamp
    
# Get run number
def getRun(file):

    buff=file.split('_')
    run = int(buff[-2].lstrip("run"))

    return run

############################### DATA IMPORTER FUNCTIONS ###############################

# Load a single file
def getDataFrame(file, offchs, timeseries=True, run=True):
   
    df=pd.read_csv(file, sep=',')
    
    # remove list of PMTs that are off
    df= df[~df['pmt'].isin(offchs)]
    
    if timeseries:
        df["timestamp"] = getTimestamp(file)
        df['date'] = pd.to_datetime(df['timestamp'], unit='s')
        df.set_index("timestamp", inplace=True)
        
    if run:
        df["run"] = getRun(file)
    
    return df
    
def dataLoader( offchPMTs, sourcedir = "../calibrationdb/", interval=(1610067905, 1637788392)):
        
    print("Import data in folder{} for interval ({}:{})".format(sourcedir, interval[0], interval[1]))
    
    # Load the data from the fit database
    data = pd.concat([ getDataFrame(sourcedir+file, offchPMTs) for file in  os.listdir(sourcedir) if "backgroundphotons" in file ])
    # keep data only for the selected interval
    data = data.loc[(data.index>=interval[0]) & (data.index<interval[1])]  
    # Sort the indeces by time
    data = data.sort_index()
    
    return data

###############################

def fitGains(x):
    BINSIZE=0.005; RMIN=0.0; RMAX=1.0; p0=(0.4, 0.04);
    nbins=int((RMAX-RMIN)/BINSIZE)
    ys,edges = np.histogram(x,bins=nbins,range=(RMIN,RMAX))
    xs=np.array([edges[i]+0.5*(edges[i+1]-edges[i]) for i in range(len(ys))])

    param = [np.max(ys), p0[0], p0[1] ]
    bounds = np.array([(param[0]*0.5,param[0]*1.5),
              (param[1]*0.2,param[1]*1.3),
              (param[2]*0.1,param[2]*1.4)])
    
    param,pcov = curve_fit(gaus, xs, ys, p0=param, bounds=(bounds[:,0],bounds[:,1]) )
    eparam = np.diag(pcov)**0.5
    equal, eEqual = getEqualization( param[1], param[2], param[2], eparam[2] )
    
    return param[1], param[2], equal, eEqual # returns mean_q, sigma, equal, eEqual

# Getting data for plotting

In [ ]:
# Define source parameters 
sdir   = "/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"

# Import data between December 2022 and December 2023
trange = (1671100000, 1702598400)
# Import data between August 2021 and November 2021
# trange = (1627585983, 1636156757) # for pre-OB loss
# Import all available historical data
# trange = (1610067904,1702598400)

offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
offchs = PMTid_to_channel(offPMTs)

data = dataLoader( offchs, sdir, interval= trange)

In [ ]:
# drop unneeded colums
data.drop(columns=['chi2', 'fitstatus','ndf','amplitude','eamplitude','sigma','esigma','mu','emu'], inplace=True)
data.reset_index(inplace=True)
data["cryo"] = data.pmt.apply(lambda x: getCryo(x))

dplot = data.groupby(["timestamp","date","run"]).agg(list).reset_index()
dplot[["mean_fit_q","fit_sigma","equal","eEqual"]] = dplot.apply(lambda x: fitGains(x["q"]), axis=1, result_type='expand')

In [ ]:
dplot.head()

## Mean gain over time

In [ ]:
fig = plt.figure(figsize=(12, 4.8))

# total gain loss
qstart = dplot.loc[dplot['timestamp'].idxmin(), 'mean_fit_q']
qend = dplot.loc[dplot['timestamp'].idxmax(), 'mean_fit_q']
diff = qend-qstart
perc = diff/qstart*100

param,pcov = curve_fit(line, xdata=dplot.timestamp.to_numpy(), ydata=dplot.mean_fit_q.to_numpy(), sigma=dplot.fit_sigma.to_numpy())
ys = line(dplot.timestamp.to_numpy(),param[0],param[1])

month_in_s = 30*24*60*60
day_in_s = 24*60*60
month_loss = param[0]*month_in_s/qstart*100
day_loss = param[0]*day_in_s/qstart*100
print("Monthly loss: {:.4f}%".format(month_loss))
print("Daily loss: {:.4f}%".format(day_loss))

plt.plot( dplot.date.to_numpy(), ys, color="red", lw=2, label="Linear fit:\n$-${:.2f}%/month".format(-1*month_loss))
plt.errorbar( x=dplot.date, y=dplot.mean_fit_q, yerr=dplot.fit_sigma, fmt='o',label="Mean PMT gain\nTotal loss: {:.2f}%".format(perc))

for i,r in enumerate(dplot.run.to_numpy()):
    plt.annotate( str(r), xy=(dplot.date.to_numpy()[i],dplot.mean_fit_q.to_numpy()[i]),textcoords="offset points", xytext=(0,22), ha='center')

plt.ylim((0.4,0.5))
plt.ylabel("Gain [$10^7$ electrons]", fontsize=12)
plt.title("ICARUS Run 2 - Mean PMT gain", fontsize=14)
plt.grid(alpha=0.5, linestyle="dashed")
plt.setp(plt.gca().get_xticklabels(), rotation=60, ha="right", fontsize=12)
plt.legend(fontsize=12)

plt.savefig("figs/ICARUS-Run2_Mean_PMT_gain.png",dpi=600)
plt.show()

## Equalization over time

In [ ]:
fig = plt.figure(figsize=(12, 4.8))

plt.errorbar( x=dplot.date, y=dplot.equal*100, yerr=dplot.eEqual*100, fmt='o')
for i,r in enumerate(dplot.run.to_numpy()):
    plt.annotate( str(r), xy=(dplot.date.to_numpy()[i],dplot.equal.to_numpy()[i]*100),textcoords="offset points", xytext=(0,18), ha='center')

plt.ylim((1.3,2.3))
plt.ylabel("Equalization [%]", fontsize=12)
plt.title("ICARUS Run 2 - PMT Equalization", fontsize=14)
plt.grid(alpha=0.5, linestyle="dashed")
plt.setp(plt.gca().get_xticklabels(), rotation=60, ha="right", fontsize=12)
#plt.legend(fontsize=12)

plt.savefig("figs/ICARUS-Run2_PMT_equalization.png",dpi=600)
plt.show()

## Single PMTs gain variation

In [ ]:
dpmt = data.groupby(["pmt","cryo"]).agg(list).reset_index()
dpmt.head()

In [ ]:
# select channels
channels = [0, 149, 185]

fig = plt.figure(figsize=(12, 4.8))
for ch in range(20,25): #in channels:
    
    if ch in offchs:
        continue
    _sel = dpmt.pmt == ch
    
    xs=dpmt[_sel].date.to_numpy()[0]
    ys=dpmt[_sel].q.to_numpy()[0]
    err=dpmt[_sel]['eq'].to_numpy()[0]
        
    plt.errorbar( x=xs, y=ys, yerr=err, fmt='o-', label='PMT ID {}'.format(channel_to_PMTid(ch)))
    for i,r in enumerate(dpmt[_sel].run.to_numpy()[0]):
        plt.annotate( str(r), xy=(xs[i],ys[i]),textcoords="offset points", xytext=(0,18), ha='center')

plt.ylabel("Gain [$10^7$ electrons]", fontsize=12)
plt.title("ICARUS Run 2 - PMT Gains", fontsize=14)
plt.grid(alpha=0.5, linestyle="dashed")
plt.setp(plt.gca().get_xticklabels(), rotation=60, ha="right", fontsize=12)
plt.legend()

#plt.savefig("figs/ICARUS-Run2_PMT_equalization.png",dpi=600)
plt.show()

## Gain by y quota

In [ ]:
dpmt["y"] = dpmt.pmt.apply(lambda x: getY(x))
dyquota = dpmt.explode(["timestamp", "q","date","run"]).reset_index()
dyquota.drop(columns=["index","nentries","eq"],inplace=True)

In [ ]:
dy = dyquota.groupby(["timestamp","date","run","y"]).agg(list)
dy[["mean_fit_q","fit_sigma","equal","eEqual"]] = dy.apply(lambda x: fitGains(x["q"]), axis=1, result_type='expand')
dy.reset_index(inplace=True)
dy.drop(columns=["pmt","cryo","q","equal","eEqual"],inplace=True)
dy = dy.groupby(["y"]).agg(list).reset_index()
dy.head()

In [ ]:
fig = plt.figure(figsize=(12, 4.8))

for quota in (-129.05,-76.25,-23.45,29.35,82.15):
    
    _sel = dy.y == quota
    xs=dy[_sel].date.to_numpy()[0]
    ys=dy[_sel].mean_fit_q.to_numpy()[0]
    err=dy[_sel].fit_sigma.to_numpy()[0]
    plt.errorbar( x=xs, y=ys, yerr=err, fmt='o',label="y$=${}cm".format(quota))

for i,r in enumerate(dy[_sel].run.to_numpy()[0]):
    plt.annotate( str(r), xy=(xs[i],ys[i]),textcoords="offset points", xytext=(0,25), ha='center')

plt.ylim((0.4,0.5))
plt.ylabel("Gain [$10^7$ electrons]", fontsize=12)
plt.title("ICARUS Run 2 - Mean PMT gain", fontsize=14)
plt.grid(alpha=0.5, linestyle="dashed")
plt.setp(plt.gca().get_xticklabels(), rotation=60, ha="right", fontsize=12)
plt.legend(fontsize=12)

plt.show()

## Gain by cryostat

In [ ]:
# WEST
dw = data[data.cryo==1].groupby(["timestamp","date","run"]).agg(list).reset_index()
dw[["mean_fit_q","fit_sigma","equal","eEqual"]] = dw.apply(lambda x: fitGains(x["q"]), axis=1, result_type='expand')
# EAST
de = data[data.cryo==0].groupby(["timestamp","date","run"]).agg(list).reset_index()
de[["mean_fit_q","fit_sigma","equal","eEqual"]] = de.apply(lambda x: fitGains(x["q"]), axis=1, result_type='expand')

In [ ]:
fig = plt.figure(figsize=(12, 4.8))

plt.errorbar( x=dw.date, y=dw.mean_fit_q, yerr=dw.fit_sigma, fmt='o',label="WEST")
plt.errorbar( x=de.date, y=de.mean_fit_q, yerr=de.fit_sigma, fmt='o',label="EAST")

for i,r in enumerate(dw.run.to_numpy()):
    plt.annotate( str(r), xy=(dw.date.to_numpy()[i],dw.mean_fit_q.to_numpy()[i]),textcoords="offset points", xytext=(0,22), ha='center')

#plt.ylim((0.4,0.5))
plt.ylabel("Gain [$10^7$ electrons]", fontsize=12)
plt.title("ICARUS Run 2 - Mean PMT gain", fontsize=14)
plt.grid(alpha=0.5, linestyle="dashed")
plt.setp(plt.gca().get_xticklabels(), rotation=60, ha="right", fontsize=12)
plt.legend(fontsize=12)

plt.show()